In [83]:
import pandas as pd
import numpy as np
import mibian
import math
from scipy.stats import norm
import matplotlib.pyplot as plt
import statsmodels.api as sm 
import py_vollib 
#from py_vollib.black_scholes.implied_volatility import implied_volatility as iv
import py_vollib.black_scholes.implied_volatility as biv
#import py_vollib.black.implied_volatility as biv
from py_vollib.black_scholes import black_scholes as bs

In [93]:
#OptionList = pd.read_csv("nifty_311221_070122_1min.csv",index_col=0)
OptionList = pd.read_csv("nifty_311221_190122_1min.csv",index_col=0)

In [100]:
test_set = OptionList[OptionList['instrument_type']=='CE'].iloc[:100,:]
test_set.count()

date               100
opt_price          100
volume_x           100
opt_symbol         100
fut_price          100
volume_y           100
fut_symbol         100
name               100
expiry             100
strike             100
instrument_type    100
ts                 100
days_to_expiry     100
dtype: int64

In [33]:
# see reference: https://github.com/yassinemaaroufi/MibianLib
def calculateIV(row):
    s = row['fut_price']
    k = row['strike']
    r = 0
    t = row['days_to_expiry']
    price = row['opt_price']
    if row['instrument_type'] == 'CE':
        bs = mibian.BS([s,k,r,t],callPrice=price)
    elif row['instrument_type'] == 'PE':
        bs = mibian.BS([s,k,r,t],putPrice=price)
    return bs.impliedVolatility

def calculateDelta(row):
    s = row['fut_price']
    k = row['strike']
    r = 0
    t = row['days_to_expiry']
    v = row['IV']
    bs = mibian.BS([s,k,r,t],volatility=v)
    if row['instrument_type'] == 'CE':
        res = bs.callDelta
    elif row['instrument_type'] == 'PE':
        res = bs.putDelta
    return res

def calculateVega(row):
    s = row['fut_price']
    k = row['strike']
    r = 0
    t = row['days_to_expiry']
    v = row['IV']
    bs = mibian.BS([s,k,r,t],volatility=v)
    return bs.vega

In [50]:
def calculateIV_py_vollib(row):
    s = row['fut_price']
    k = row['strike']
    r = 0
    t = row['days_to_expiry']/252
    price = row['opt_price']
    if row['instrument_type'] == 'CE':
        iv = biv.implied_volatility(price, s, k, t, r, 'c')
    elif row['instrument_type'] == 'PE':
        iv = biv.implied_volatility(price, s, k, t, r, 'p')
    return iv

In [71]:
%%time
row = test_set.iloc[1]
s = row['fut_price']
k = row['strike']
r = 0
flag = 'c'
t = row['days_to_expiry']/252
price = row['opt_price']
iv = biv.implied_volatility(price, s, k, t, r, flag)
bs_price = bs('c',row['fut_price'],row['strike'],row['days_to_expiry']/252,0,iv)
print("py_vollib iv: ",iv)
print("py_vollib call price based on implied iv: ", bs_price)
print("market option price: ", row['opt_price'])

py_vollib iv:  0.14855078758908924
py_vollib call price based on implied iv:  544.95
market option price:  544.95
Wall time: 0 ns


In [68]:
%%time
row = test_set.iloc[1]
iv = calculateIV(row)
c = mibian.BS([row['fut_price'],row['strike'],0,row['days_to_expiry']],volatility = iv)
print("mibian iv: ",iv)
print("mibian call price based on mibian iv: ", c.callPrice)
print("market option price: ", row['opt_price'])

mibian iv:  17.878055572509766
mibian call price based on mibian iv:  544.9494134124106
market option price:  544.95
Wall time: 10 ms


In [72]:
def bs_call(S, K, T, r, vol):
    d1 = (np.log(S/K) + (r + 0.5*vol**2)*T) / (vol*np.sqrt(T))
    d2 = d1 - vol * np.sqrt(T)
    return S * norm.cdf(d1) - np.exp(-r * T) * K * norm.cdf(d2)

In [74]:
bs_call(row['fut_price'],row['strike'],row['days_to_expiry']/252,0,iv)

544.9499999999989

In [69]:
iv

17.878055572509766

In [103]:
test1 = test_set[test_set['fut_price'] - test_set['strike'] <= test_set['opt_price']]

In [104]:
test1['iv_py_vollib'] = test1.apply(calculateIV_py_vollib,axis=1)

In [97]:
test1.iloc[0]['iv'] = calculateIV_py_vollib(test1.iloc[0])

<ipython-input-97-8520272cab29>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test1.iloc[0]['iv'] = calculateIV_py_vollib(test1.iloc[0])
D:\Anaconda3\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [105]:
test1

,date,opt_price,volume_x,opt_symbol,fut_price,volume_y,fut_symbol,name,expiry,strike,instrument_type,ts,days_to_expiry,iv_py_vollib
0,2021-12-31 09:15:00+05:30,231.50,0.0,NIFTY22JAN17450CE,17311.0,174100.0,NIFTY22JANFUT,NIFTY,2022-01-27 15:30:30,17450.0,CE,2021-12-31 09:15:00,27.260764,0.129723
1,2021-12-31 09:15:00+05:30,544.95,0.0,NIFTY22JAN16950CE,17311.0,174100.0,NIFTY22JANFUT,NIFTY,2022-01-27 15:30:30,16950.0,CE,2021-12-31 09:15:00,27.260764,0.148551
5,2021-12-31 09:15:00+05:30,60.50,0.0,NIFTY22JAN17950CE,17311.0,174100.0,NIFTY22JANFUT,NIFTY,2022-01-27 15:30:30,17950.0,CE,2021-12-31 09:15:00,27.260764,0.115278
6,2021-12-31 09:15:00+05:30,50.70,0.0,NIFTY22JAN18050CE,17311.0,174100.0,NIFTY22JANFUT,NIFTY,2022-01-27 15:30:30,18050.0,CE,2021-12-31 09:15:00,27.260764,0.119119
7,2021-12-31 09:15:00+05:30,4.25,0.0,NIFTY22JAN19600CE,17311.0,174100.0,NIFTY22JANFUT,NIFTY,2022-01-27 15:30:30,19600.0,CE,2021-12-31 09:15:00,27.260764,0.167532
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
176,2021-12-31 09:16:00+05:30,59.70,22500.0,NIFTY22JAN18000CE,17332.1,191750.0,NIFTY22JANFUT,NIFTY,2022-01-27 15:30:30,18000.0,CE,2021-12-31 09:16:00,27.260069,0.117820
177,2021-12-31 09:16:00+05:30,9.30,100.0,NIFTY22JAN18700CE,17332.1,191750.0,NIFTY22JANFUT,NIFTY,2022-01-27 15:30:30,18700.0,CE,2021-12-31 09:16:00,27.260069,0.124858
178,2021-12-31 09:16:00+05:30,1355.00,650.0,NIFTY22JAN16000CE,17332.1,191750.0,NIFTY22JANFUT,NIFTY,2022-01-27 15:30:30,16000.0,CE,2021-12-31 09:16:00,27.260069,0.157566
179,2021-12-31 09:16:00+05:30,22.95,6400.0,NIFTY22JAN18300CE,17332.1,191750.0,NIFTY22JANFUT,NIFTY,2022-01-27 15:30:30,18300.0,CE,2021-12-31 09:16:00,27.260069,0.115218


In [106]:
Calls = OptionList[OptionList['instrument_type']=='CE']
Calls = Calls[Calls['fut_price'] - Calls['strike'] <= Calls['opt_price']]

In [107]:
%%time
#Calls['IV'] = Calls.apply(calculateIV,axis=1)
Calls['IV'] = Calls.apply(calculateIV_py_vollib,axis=1)
Calls['delta'] = Calls.apply(calculateDelta,axis=1)
Calls['vega'] = Calls.apply(calculateVega,axis=1)
Calls.to_csv('nifty_311221_190122_1min_calls.csv')

Wall time: 12min 10s
